# Swimming Pool Detection (STDL) running on FHNW HPC
## Version 2021-01-13
Model using combined Data of Geneva and Neuchatel
Datasets were formatted into standardized COCO and merged with datumaro

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">


# Install dependencies



In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available(), torch.cuda.get_device_name(0))
!gcc --version
# opencv is pre-installed on colab

In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.5")
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
import pickle

# Train on a custom dataset

In [ ]:
#Change to Directory with combined dataset
os.chdir("../output-comb")

In [ ]:
#! rm -rf {trn,tst,val}-images-256 && tar xfvz /content/drive/My\ Drive/DeepLearning/SwimmingPoolDetection_NE/1/images-256.tar.gz > /dev/null


## Fetch the dataset from Google Drive and unpack it

In [ ]:
! ls trn-images-256 | wc -l
! ls val-images-256 | wc -l
! ls tst-images-256 | wc -l

Register the swimming pool dataset to Detectron2, following the [detectron2 custom dataset tutorial](https://detectron2.readthedocs.io/tutorials/datasets.html).


In [ ]:
COCO_TRN_FILE = "instances_trn.json"
COCO_VAL_FILE = "instances_val.json"
COCO_TST_FILE = "instances_tst.json"

# with open(COCO_TRN_FILE, 'r') as fp:
#   coco_trn_dict = json.load(fp)

from detectron2.data.datasets import register_coco_instances
register_coco_instances("swimmingpool_trn_dataset", {}, COCO_TRN_FILE, "")
register_coco_instances("swimmingpool_val_dataset", {}, COCO_VAL_FILE, "")
register_coco_instances("swimmingpool_tst_dataset", {}, COCO_TST_FILE, "")

In [ ]:
# let's check that everything's fine
#DatasetCatalog.get("swimmingpool_trn_dataset")[0]
MetadataCatalog.get("swimmingpool_trn_dataset")

To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:



In [ ]:
import matplotlib.pyplot as plt

for d in random.sample(DatasetCatalog.get("swimmingpool_trn_dataset"), 4):
    print(d["file_name"])
    output_filename = "tagged_" + d["file_name"].split('/')[-1]
    output_filename = output_filename.replace('tif', 'png')
    print(output_filename)
    img = cv2.imread(d["file_name"])  
    visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get("swimmingpool_trn_dataset"), scale=1.0)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :, :])
    cv2.imwrite(output_filename, vis.get_image()[:, :, ::-1])

...as well as randomly selected samples in the validation set:

In [ ]:
for d in random.sample(DatasetCatalog.get("swimmingpool_val_dataset"), 3):
    print(d["file_name"])
    img = cv2.imread(d["file_name"])  
    visualizer = Visualizer(img, metadata=MetadataCatalog.get("swimmingpool_val_dataset"), scale=1.0)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :, ::-1])

...and, finally, in the test test:

In [ ]:
for d in random.sample(DatasetCatalog.get("swimmingpool_tst_dataset"), 3):
    print(d["file_name"]) 
    img = cv2.imread(d["file_name"])  
    visualizer = Visualizer(img, metadata=MetadataCatalog.get("swimmingpool_tst_dataset"), scale=1.0)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :, ::-1])

## Train!

Now, let's fine-tune a coco-pretrained R50-FPN Mask R-CNN model on the swimmingpool dataset.


In [ ]:
# cf. https://medium.com/@apofeniaco/training-on-detectron2-with-a-validation-set-and-plot-loss-on-it-to-avoid-overfitting-6449418fbf4e
# cf. https://towardsdatascience.com/face-detection-on-custom-dataset-with-detectron2-and-pytorch-using-python-23c17e99e162
# cf. http://cocodataset.org/#detection-eval


import time
import datetime
import logging
import detectron2.utils.comm as comm

from detectron2.utils.logger import log_every_n_seconds
from detectron2.engine.hooks import HookBase
from detectron2.data import DatasetMapper, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultTrainer

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):

        #print('Entering here...')

        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):

        #print('Entering there...')

        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):

        #print('Entering overthere...')

        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)



class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("COCO_eval", exist_ok=True)
        output_folder = "COCO_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

  
  def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks

In [ ]:
LOG_DIR = 'runs' 
os.makedirs(LOG_DIR, exist_ok=True)

! rm -rf {LOG_DIR}/run*

runs = [] #[1,2,3,4,5,6,7,8,9]

In [ ]:
from detectron2.config import get_cfg

# cf. https://detectron2.readthedocs.io/modules/config.html#config-references
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = ("swimmingpool_trn_dataset",)
cfg.DATASETS.TEST = ("swimmingpool_val_dataset", ) #No evaluator found. Use `DefaultTrainer.test(evaluators=)`, or implement its `build_evaluator` method.
cfg.TEST.EVAL_PERIOD = 200
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2 # Number of images processed in one iteration
cfg.SOLVER.BASE_LR = 0.005 # 0.000025 #*5  # pick a good LR
cfg.SOLVER.CHECKPOINT_PERIOD = 1000  # Save
cfg.SOLVER.STEPS = (2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500)
cfg.SOLVER.GAMMA = 0.8
cfg.SOLVER.WARMUP_ITERS = 200
cfg.SOLVER.MAX_ITER = 1000 # it seems to be the "sweet spot" ;-)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024  # perhaps faster, to be checked (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class ("swimming pool")

# Size of the smallest side of the image during training
#cfg.INPUT.MIN_SIZE_TRAIN = (256,) # (640, 672, 704, 736, 768, 800)
# Maximum size of the side of the image during training
#cfg.INPUT.MAX_SIZE_TRAIN = 256 # 1333
# Size of the smallest side of the image during testing. Set to zero to disable resize in testing.
#cfg.INPUT.MIN_SIZE_TEST = 0 #800
# Maximum size of the side of the image during testing
#cfg.INPUT.MAX_SIZE_TEST = 0 #1333

cfg.INPUT.FORMAT = "RGB"

#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

cfg.MODEL.MASK_ON = True

In [ ]:
cfg.INPUT.CROP

In [ ]:
resume=False # TO DO: make resume work!

if not resume:
  # create a new run  
  runs.append(len(runs))

  cfg.OUTPUT_DIR = os.path.join(LOG_DIR, f'run{len(runs)}')
  os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

  filelist = [ f for f in os.listdir(cfg.OUTPUT_DIR) ] #if f.endswith(".bak") ]
  for f in filelist:
    os.remove(os.path.join(cfg.OUTPUT_DIR, f))

In [ ]:
cfg.OUTPUT_DIR

In [ ]:
#trainer = DefaultTrainer(cfg)
import torch
import torchvision
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=resume)
trainer.train()

In [ ]:
with open(os.path.join(cfg.OUTPUT_DIR, 'cfg.yaml'), 'w') as fp:
  fp.write(cfg.dump())

! rm {cfg.OUTPUT_DIR}/model.tar.gz
! tar -czvf {cfg.OUTPUT_DIR}/model.tar.gz {cfg.OUTPUT_DIR}/model_final.pth {cfg.OUTPUT_DIR}/metrics.json {cfg.OUTPUT_DIR}/cfg.yaml
! cp {cfg.OUTPUT_DIR}/model.tar.gz ../../SwimmingPoolDetection_NE/2

In [ ]:
from detectron2.modeling import build_model
model = build_model(cfg)

def count_parameters(model, trainable_only=False):

  if trainable_only:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
  else:
    return sum(p.numel() for p in model.parameters())


print(f'{count_parameters(model, trainable_only=False)} parameters in this model, {count_parameters(model, trainable_only=True)} of which are trainable')


In [ ]:
#! cp {cfg.OUTPUT_DIR}/model_final.pth /content/drive/My\ Drive/DeepLearning/SwimmingPoolDetection_NE/1

## Inference & evaluation using the trained model
Now, let's run inference with the trained model on the validation dataset. First, let's create a predictor using the model we just trained:



In [ ]:
cfg.OUTPUT_DIR

In [ ]:
! ls {cfg.OUTPUT_DIR}

In [ ]:
! cp ../../SwimmingPoolDetection_NE/1/model.tar.gz .
! tar xzfv model.tar.gz 
! ls 

In [ ]:
# >>> this allows us to load a previously trained model <<<
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8 # set the testing threshold for this model, cf. https://detectron2.readthedocs.io/modules/config.html#config-references
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [ ]:
from detectron2.utils.visualizer import ColorMode

for d in random.sample(DatasetCatalog.get("swimmingpool_tst_dataset"), 10):
#for d in DatasetCatalog.get("swimmingpool_tst_dataset")[10:18]:   
    print(d["file_name"])
    output_filename = "pred_" + d["file_name"].split('/')[-1]
    output_filename = output_filename.replace('tif', 'png')
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], # [:, :, ::-1] is for RGB -> BGR conversion, cf. https://stackoverflow.com/questions/14556545/why-opencv-using-bgr-colour-space-instead-of-rgb
                   metadata=MetadataCatalog.get("swimmingpool_tst_dataset"), 
                   scale=1.0#, 
                   #instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    #print(dir(v))
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(v.get_image()[:, :, :])
    cv2.imwrite(output_filename, v.get_image()[:, :, ::-1])

In [ ]:
# HELPER FUNCTIONS

def _preprocess(preds):
  
  fields = preds['instances'].get_fields()

  out = {}

  # pred_boxes
  if 'pred_boxes' in fields.keys():
    out['pred_boxes'] = [box.cpu().numpy() for box in fields['pred_boxes']]
  # pred_classes
  if 'pred_classes' in fields.keys():
    out['pred_classes'] = fields['pred_classes'].cpu().numpy()
  # pred_masks
  if 'pred_masks' in fields.keys():
    out['pred_masks'] = fields['pred_masks'].cpu().numpy()
  # scores
  if 'scores' in fields.keys():
    out['scores'] = fields['scores'].cpu().numpy()

  return out


def dt2predictions_to_list(preds):

  instances = []
  
  tmp = _preprocess(preds)

  for idx in range(len(tmp['scores'])):
    instance = {}
    instance['score'] = tmp['scores'][idx]
    instance['pred_class'] = tmp['pred_classes'][idx]

    if 'pred_masks' in tmp.keys():
      instance['pred_mask'] = tmp['pred_masks'][idx]
    
    instance['pred_box'] = tmp['pred_boxes'][idx]
    
    instances.append(instance)

  return instances

In [ ]:
%%time

# Let's make predictions over the entire test set - at fixed TEST THRESHOLD

OUTPUT_IMG_DIR = "predictions-256"
os.makedirs(OUTPUT_IMG_DIR, exist_ok=True)

predictions_dict = {}

# remove already existing files
filelist = [ f for f in os.listdir(OUTPUT_IMG_DIR) ] #if f.endswith(".bak") ]
for f in filelist:
    os.remove(os.path.join(OUTPUT_IMG_DIR, f))

#for d in random.sample(DatasetCatalog.get("swimmingpool_val_dataset"), 3):
for d in DatasetCatalog.get("swimmingpool_tst_dataset"):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    predictions_dict[d['file_name']] = dt2predictions_to_list(outputs)
    
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("swimmingpool_tst_dataset"), 
                   scale=1.0#, 
                   #instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    #cv2_imshow(v.get_image())
    
    output_filename = os.path.join(OUTPUT_IMG_DIR, d['file_name'].split('/')[-1])
    cv2.imwrite(output_filename, v.get_image()[:, :, ::-1])


with open(f'predictions_at_fixed_threshold_dict.pkl', 'wb') as fp:
  pickle.dump(predictions_dict, fp)

In [ ]:
! rm predictions-256.tar.gz
! tar -czvf predictions-256.tar.gz predictions-256/ > /dev/null
! cp predictions-256.tar.gz ../../SwimmingPoolDetection_NE/1

! cp predictions_at_fixed_threshold_dict.pkl ../../SwimmingPoolDetection_NE/1

We can evaluate its performance using the AP metric implemented in COCO API.

cf. https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# the following evaluation DOES depend on the SCORE_THRESH_TEST
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05

evaluator = COCOEvaluator("swimmingpool_tst_dataset", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "swimmingpool_tst_dataset")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test

In [ ]:
%%time

# predictions as a function of the THRESHOLD
# N.B.: the following is not actually needed, as predictions @ any threshold can be obtained 
#       from the predictions @ threshold = 0.05 by filtering on the score 

# from tqdm.notebook import tqdm

# predictions_at_various_thresholds = {}

# for el in tqdm(np.arange(0.05, 1., 0.05)):
#   threshold = round(float(el),2)
#   print(f'threshold={threshold}')
#   cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold   # set the testing threshold for this model
#   predictor = DefaultPredictor(cfg)

#   predictions_at_various_thresholds[threshold] = {}

#   for d in DatasetCatalog.get("swimmingpool_val_dataset"):
#       im = cv2.imread(d["file_name"])
#       outputs = predictor(im)  
#       predictions_at_various_thresholds[threshold][d['file_name']] = dt2predictions_to_list(outputs)

# with open('predictions_at_various_thresholds_dict.pkl', 'wb') as fp:
#   pickle.dump(predictions_at_various_thresholds, fp)

In [ ]:
threshold = 0.05
threshold_str = str( round(threshold, 2) ).replace('.', 'dot')

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)

trn_predictions = {}
val_predictions = {}
tst_predictions = {}

for d in DatasetCatalog.get("swimmingpool_trn_dataset"):
  im = cv2.imread(d["file_name"])
  outputs = predictor(im)  
  trn_predictions[d['file_name']] = dt2predictions_to_list(outputs)

with open(f'trn_predictions_at_{threshold_str}_threshold.pkl', 'wb') as fp:
  pickle.dump(trn_predictions, fp)

del trn_predictions

for d in DatasetCatalog.get("swimmingpool_val_dataset"):
  im = cv2.imread(d["file_name"])
  outputs = predictor(im)  
  val_predictions[d['file_name']] = dt2predictions_to_list(outputs)

with open(f'val_predictions_at_{threshold_str}_threshold.pkl', 'wb') as fp:
  pickle.dump(val_predictions, fp)

del val_predictions

for d in DatasetCatalog.get("swimmingpool_tst_dataset"):
  im = cv2.imread(d["file_name"])
  outputs = predictor(im)  
  tst_predictions[d['file_name']] = dt2predictions_to_list(outputs)

with open(f'tst_predictions_at_{threshold_str}_threshold.pkl', 'wb') as fp:
  pickle.dump(tst_predictions, fp)

del tst_predictions

print("...done.")

In [ ]:
! cp ???_predictions_at_{threshold_str}_threshold.pkl ../../SwimmingPoolDetection_NE/1

In [ ]:
! ls ../../SwimmingPoolDetection_NE/1

## Making predictions over the entire tile set

In [ ]:
stop here

In [ ]:
#! rm -rf all-images-256 && tar xfvz /content/drive/My\ Drive/DeepLearning/SwimmingPoolDetection_NE/1/all-images-256.tar.gz > /dev/null
! rm -rf all-images-256 && cp /content/drive/My\ Drive/DeepLearning/SwimmingPoolDetection_NE/1/all-images-256.tar.gz . && tar xfvz all-images-256.tar.gz > /dev/null


In [ ]:
! ls all-images-256 | wc -l

In [ ]:
# the file all-images-256/18_135968_92299.tif was once found to be corrupted
# before launching the following code block, make sure that there is no TIF file smaller than 100k
! find all-images-256/*.tif -type f -size -100k

In [ ]:
from tqdm.notebook import tqdm

threshold = 0.05
threshold_str = str( round(threshold, 2) ).replace('.', 'dot')

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)

all_predictions = {}

tic = time.time()

img_files = [x for x in os.listdir("all-images-256") if ".tif" in x]

for file_name in tqdm(img_files):

  im = cv2.imread("all-images-256/" + file_name)
  outputs = predictor(im)  
  all_predictions["all-images-256/" + file_name] = dt2predictions_to_list(outputs)

toc = time.time()

with open(f'all_predictions_at_{threshold_str}_threshold.pkl', 'wb') as fp:
  pickle.dump(all_predictions, fp)

del all_predictions

print(f"...done in {(toc-tic)/60} min.")

In [ ]:
! cp all_predictions_at_{threshold_str}_threshold.pkl ../../SwimmingPoolDetection_NE/1

In [ ]:
import tqdm
for i in tqdm(range(10)):
    time.sleep(1)